#  DataCamp Blog Scraper

## 🎯 Призначення
Python скрипт для автоматичного збору статей з блогу DataCamp (datacamp.com/blog).

## 🔧 Основні функції

### **1. `scrape_datacamp_alternative()` — RSS парсинг**
- **Метод:** Завантаження RSS feed (https://www.datacamp.com/blog/rss.xml)
- **Переваги:** Найнадійніший, офіційний спосіб
- **Дані:** Заголовок, URL, опис, дата публікації
- **Результат:** DataFrame з 4 колонками

### **2. `scrape_datacamp_working()` — HTML парсинг**
- **Метод:** 3 послідовні стратегії пошуку
  - Метод 1: Пошук `<article>` тегів
  - Метод 2: Пошук `<div>` з класами (post, card, item)
  - Метод 3: Загальний пошук посилань `/blog/`
- **Переваги:** Fallback якщо RSS недоступний
- **Дані:** Заголовок, URL
- **Результат:** DataFrame з 2 колонками

### **3. `main()` — Головний orchestrator**
- Пробує спочатку RSS (найкращий метод)
- Якщо RSS fails → переходить на HTML парсинг
- Зберігає результати в CSV файли
- Виводить статистику та перші 5 статей

## 📦 Технології
- **requests** — HTTP запити
- **BeautifulSoup4** — HTML/XML парсинг
- **pandas** — обробка даних
- **lxml** — XML parser для RSS


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import json

def scrape_datacamp_working():
    """
    Робоча версія парсера DataCamp Blog
    """
    url = "https://www.datacamp.com/blog"
    
    # Більш реалістичні headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Cache-Control': 'max-age=0'
    }
    
    try:
        print("🔍 Відправляємо запит до DataCamp Blog...")
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        print(f"✅ Отримано відповідь: {response.status_code}")
        print(f"📄 Розмір сторінки: {len(response.text)} символів")
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        articles = []
        
        # Метод 1: Шукаємо article tags
        article_tags = soup.find_all('article')
        print(f"🔎 Знайдено <article> тегів: {len(article_tags)}")
        
        for article in article_tags:
            # Шукаємо заголовок
            title_tag = article.find(['h2', 'h3', 'h4'], class_=lambda x: x and ('title' in x.lower() or 'heading' in x.lower()))
            
            if not title_tag:
                title_tag = article.find(['h2', 'h3', 'h4'])
            
            # Шукаємо посилання
            link_tag = article.find('a', href=True)
            
            if title_tag and link_tag:
                title = title_tag.get_text(strip=True)
                url = link_tag.get('href')
                
                # Формуємо повний URL якщо потрібно
                if url.startswith('/'):
                    url = f"https://www.datacamp.com{url}"
                
                if len(title) > 10 and '/blog/' in url:
                    articles.append({
                        'title': title,
                        'url': url
                    })
        
        # Метод 2: Якщо article tags не спрацювали, шукаємо div з класами
        if len(articles) == 0:
            print("⚠️ Article tags не знайдено, пробуємо інший підхід...")
            
            # Шукаємо всі div з можливими класами блогу
            blog_items = soup.find_all('div', class_=lambda x: x and any(
                keyword in x.lower() for keyword in ['post', 'card', 'item', 'article']
            ))
            
            print(f"🔎 Знайдено blog items: {len(blog_items)}")
            
            for item in blog_items:
                link = item.find('a', href=True)
                if link and '/blog/' in link.get('href'):
                    title_elem = item.find(['h2', 'h3', 'h4', 'h5'])
                    
                    if title_elem:
                        title = title_elem.get_text(strip=True)
                        url = link.get('href')
                        
                        if url.startswith('/'):
                            url = f"https://www.datacamp.com{url}"
                        
                        if len(title) > 10:
                            articles.append({
                                'title': title,
                                'url': url
                            })
        
        # Метод 3: Шукаємо всі посилання які ведуть на /blog/
        if len(articles) == 0:
            print("⚠️ Пробуємо загальний пошук посилань...")
            
            all_links = soup.find_all('a', href=lambda x: x and '/blog/' in x and x.count('/') >= 3)
            
            print(f"🔎 Знайдено посилань на блог: {len(all_links)}")
            
            seen_urls = set()
            
            for link in all_links:
                url = link.get('href')
                
                if url.startswith('/'):
                    url = f"https://www.datacamp.com{url}"
                
                # Уникаємо дублікатів
                if url in seen_urls:
                    continue
                
                seen_urls.add(url)
                
                # Шукаємо заголовок
                title = link.get_text(strip=True)
                
                # Якщо текст посилання занадто короткий, шукаємо батьківський елемент
                if len(title) < 10:
                    parent = link.find_parent(['div', 'article', 'section'])
                    if parent:
                        title_tag = parent.find(['h2', 'h3', 'h4', 'h5'])
                        if title_tag:
                            title = title_tag.get_text(strip=True)
                
                if len(title) > 10 and len(title) < 200:
                    articles.append({
                        'title': title,
                        'url': url
                    })
        
        # Видаляємо дублікати
        unique_articles = []
        seen_urls = set()
        
        for article in articles:
            if article['url'] not in seen_urls:
                seen_urls.add(article['url'])
                unique_articles.append(article)
        
        print(f"\n✅ Знайдено унікальних статей: {len(unique_articles)}")
        
        return pd.DataFrame(unique_articles)
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Помилка при запиті: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ Несподівана помилка: {e}")
        return pd.DataFrame()


def scrape_datacamp_alternative():
    """
    Альтернативний метод: парсинг через API або RSS
    """
    # DataCamp має RSS feed
    rss_url = "https://www.datacamp.com/blog/rss.xml"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        print("🔍 Пробуємо завантажити RSS feed...")
        response = requests.get(rss_url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'xml')
        
        articles = []
        items = soup.find_all('item')
        
        print(f"✅ Знайдено статей в RSS: {len(items)}")
        
        for item in items:
            title = item.find('title').get_text(strip=True) if item.find('title') else 'No title'
            link = item.find('link').get_text(strip=True) if item.find('link') else ''
            description = item.find('description').get_text(strip=True) if item.find('description') else ''
            pub_date = item.find('pubDate').get_text(strip=True) if item.find('pubDate') else ''
            
            articles.append({
                'title': title,
                'url': link,
                'description': description,
                'published': pub_date
            })
        
        return pd.DataFrame(articles)
    
    except Exception as e:
        print(f"❌ Помилка при завантаженні RSS: {e}")
        return pd.DataFrame()


def main():
    """
    Головна функція
    """
    print("=" * 60)
    print("📰 DataCamp Blog Scraper")
    print("=" * 60)
    
    # Спочатку пробуємо RSS (найбільш надійний метод)
    print("\n🔄 Метод 1: RSS Feed")
    print("-" * 60)
    df_rss = scrape_datacamp_alternative()
    
    if len(df_rss) > 0:
        print(f"\n✅ RSS успішно! Зібрано {len(df_rss)} статей")
        df_rss.to_csv('datacamp_articles_rss.csv', index=False, encoding='utf-8')
        print("💾 Збережено в: datacamp_articles_rss.csv")
        print("\n📋 Перші 5 статей:")
        print(df_rss.head())
        return
    
    # Якщо RSS не спрацював, пробуємо HTML парсинг
    print("\n🔄 Метод 2: HTML Parsing")
    print("-" * 60)
    df_html = scrape_datacamp_working()
    
    if len(df_html) > 0:
        print(f"\n✅ HTML парсинг успішний! Зібрано {len(df_html)} статей")
        df_html.to_csv('datacamp_articles_html.csv', index=False, encoding='utf-8')
        print("💾 Збережено в: datacamp_articles_html.csv")
        print("\n📋 Перші 5 статей:")
        print(df_html.head())
    else:
        print("\n❌ Не вдалося зібрати дані жодним методом")
        print("\n💡 Можливі причини:")
        print("   1. Сайт блокує скрапінг")
        print("   2. Змінилась структура HTML")
        print("   3. Потрібен JavaScript для завантаження контенту")
        print("\n🔧 Рекомендації:")
        print("   - Використайте Selenium для dynamic контенту")
        print("   - Перевірте robots.txt: https://www.datacamp.com/robots.txt")
        print("   - Розгляньте використання офіційного API (якщо доступний)")


if __name__ == "__main__":
    main()

📰 DataCamp Blog Scraper

🔄 Метод 1: RSS Feed
------------------------------------------------------------
🔍 Пробуємо завантажити RSS feed...
❌ Помилка при завантаженні RSS: 404 Client Error: Not Found for url: https://www.datacamp.com/blog/rss.xml

🔄 Метод 2: HTML Parsing
------------------------------------------------------------
🔍 Відправляємо запит до DataCamp Blog...
✅ Отримано відповідь: 200
📄 Розмір сторінки: 414631 символів
🔎 Знайдено <article> тегів: 0
⚠️ Article tags не знайдено, пробуємо інший підхід...
🔎 Знайдено blog items: 0
⚠️ Пробуємо загальний пошук посилань...
🔎 Знайдено посилань на блог: 83

✅ Знайдено унікальних статей: 25

✅ HTML парсинг успішний! Зібрано 25 статей
💾 Збережено в: datacamp_articles_html.csv

📋 Перші 5 статей:
                 title                                                url
0        Certification  https://www.datacamp.com/blog/category/certifi...
1  DataCamp Classrooms  https://www.datacamp.com/blog/category/datacam...
2     DataCamp Donates

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_datacamp_simple():
    """Проста версія для демонстрації"""
    url = "https://www.datacamp.com/blog"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    articles = []
    
    # Знаходимо всі посилання на статті
    links = soup.find_all('a', href=True)
    
    for link in links:
        href = link.get('href')
        if '/blog/' in href and href.count('/') > 3:
            title = link.get_text(strip=True)
            if len(title) > 10:
                articles.append({
                    'title': title,
                    'url': href
                })
    
    return pd.DataFrame(articles)

# Збір даних
df = scrape_datacamp_simple()
df.to_csv('datacamp_articles.csv', index=False)
print(f"Зібрано {len(df)} статей")

Зібрано 0 статей
